In [147]:
%load_ext autoreload
%autoreload 2

from shrimpgrad.tensor import Tensor
import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [151]:
a = Tensor.arange(2)
b = Tensor.arange(2)
c = Tensor.arange(2)

t = a+b
d = t*c

d.backward()
a.grad, b.grad, c.grad, t.grad, d.grad

(tensor([0.0, 1.0]),
 tensor([0.0, 1.0]),
 tensor([0.0, 2.0]),
 tensor([0.0, 1.0]),
 tensor([1.0, 1.0]))